In [22]:
# import libraries 

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
%matplotlib inline
import random

In [23]:
# function definitions 

def derive(f,x,h):
    
    return (f(x+h) - f(x-h))/(2*h)

def derive_twice(f,x,h):
    
    return ( f(x+h) - 2*f(x) + f(x-h) ) / (h**2)

def create_interp(x,V):
    
    f = interpolate.interp1d(x,V,kind="cubic",fill_value="extrapolate")
    
    return f

In [24]:
def Verlet (x_data,E_data,i,m,x0,v0,tmax,dt):
    
    t = 0; x = x0; v = v0
    
    #V_init = create_interp(x_data,E_data[i]) 
    
    V_set = [elem for elem in E_data] 
    
    pos = []; vel = []; energy = []; time = []; pot = []; 
    
    j = i
    
    V_current = create_interp(x_data,V_set[j])
    a = -derive(V_current,x,dt)/m
    
    
    while(t<tmax):                     
            
            V_current = create_interp(x_data,V_set[j])
            
            x_tmp = x
            x = x_tmp + v*dt + 0.5*(dt)**2 * a
            
            a_tmp = a
            a = -derive(V_current,x,dt)/m
            
            v_tmp = v
            v = v_tmp + 0.5*(dt)*(a + a_tmp)
            T = 0.5*m*v**2
            
            for k in range(len(V_set)):
                
                if k == j:
                    continue
                
                V_hop = create_interp(x_data,V_set[k])
                
                V_diff = V_current(x) - V_hop(x)  
                
                dV_diff = derive(V_current,x,dt) - derive(V_hop,x,dt) 
                
            
                if dV_diff <= 1e-8: # when the derivative is effectively zero
                    
                    
                    P_LZ = np.exp( (-np.pi/2) * (np.abs(V_diff))**3/2 * (3*np.abs( derive_twice(V_current,t+dt,dt) - derive_twice(V_hop,t+dt,dt) ) )**(-0.5) )              
                    

                    num = random.uniform(0,1)

                    
                    #print(P_LZ)
                    
                    if P_LZ > num:
                        
                            
                        v_aft = np.sqrt(v**2 + (2/m)*V_diff)     # rescale the velocity (?)
                        T_aft = 0.5*v_aft**2

                        if T_aft - (V_diff + T) <= 1e-8: #check the energy gap 
                            
                            
                            j = k
                            T = T_aft
                            
                            #print("Hopping to curve: " , k)
                            index = k
                            break
                            
                        
            if x > 200:
                print("Final state: ",index )
                break
            
            t = t + dt
            
            pos.append(x)
        
            vel.append(v)
            
            pot.append(V_current(x))
           
            energy.append(V_current(x) + T)
            
            time.append(t)
            
        
# return(time,pos,vel,pot,energy)

In [25]:
# the first column contains the position

x_data = np.loadtxt("myData.txt", delimiter=' ',skiprows=2)[:,0]       

# other columns contain energy values

E_data = [np.loadtxt("myData.txt", delimiter=' ',skiprows=2)[:,i] for i in range(1,8)]

In [26]:
i = int(input("Choose the initial curve (0 for ground state): "))

Choose the initial curve (0 for ground state): 6


In [27]:
m_ccl = 1836.15*(35.453 * 12.0107)/(35.453 + 12.0107)
E = float(input("Initial kinetic energy: "))
v_ccl = (2*E/m_ccl)**0.5
x0 = -90

Initial kinetic energy: 1e3


In [ ]:
for elem in range(100):
    Verlet(x_data,E_data,i,m_ccl,x0,v_ccl,180/v_ccl,0.01)